In [458]:
#for scraping booking
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
#for scraping +api
import pandas as pd
import json
import plotly.io as pio
import plotly.express as px

# ***we work on (Paris) at the begning at the end of the page we work on all***

In [459]:
r2=requests.get('https://www.booking.com/searchresults.fr.html?ss=Le+Mont-Saint-Michel&ssne=Le+Mont-Saint-Michel&ssne_untouched=Le+Mont-Saint-Michel&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAK0iaSjBsACAdICJDI0ZTRjMGQzLTkwNjctNGJkZi04MWYzLWZhODIwNjdjZDViN9gCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=900039327&dest_type=city&group_adults=2&no_rooms=1&group_children=0')
r2.text[0:200]


"<!DOCTYPE html>\n<!--\nYou know you could be getting paid to poke around in our code?\nWe're hiring designers and developers to work in Amsterdam:\nhttps://careers.booking.com/\n-->\n<!-- wdot-802 -->\n<scri"

In [460]:
# Instanciate BeautifulSoup class
soup2=BeautifulSoup( r2.text, "html.parser") # Here we used an HTML Parser

In [461]:
#loonking for hotel links
for i in soup2.find_all('a') :
  print(i.attrs)

{'href': '#skipto_main', 'class': ['a11y-skip-to-content']}
{'href': 'https://www.booking.com/index.fr.html', 'aria-label': "Booking.com réservations d'hôtels en ligne", 'data-et-click': '\nYTBUIHOdVLBLKAMZUC:1\nYTBUIHOdVLBLKAMZUC:2\n'}
{'class': ['bui-list-item', 'bui-list-item--size-small', 'js-header-language-item'], 'href': '/city/fr/le-mont-saint-michel.en-gb.html', 'data-lang': 'en-gb', 'hreflang': 'en-gb', 'data-et-click': 'customGoal:YTTHbXeeVJWcWPaDMWOMHTccTBLWCAWdPZKe:1'}
{'class': ['bui-list-item', 'bui-list-item--size-small', 'js-header-language-item'], 'href': '/city/fr/le-mont-saint-michel.html', 'data-lang': 'en-us', 'hreflang': 'en-us', 'data-et-click': 'customGoal:YTTHbXeeVJWcWPaDMWOMHTccTBLWCAWdPZKe:1'}
{'class': ['bui-list-item', 'bui-list-item--size-small', 'js-header-language-item'], 'href': '/city/fr/le-mont-saint-michel.es.html', 'data-lang': 'es', 'hreflang': 'es', 'data-et-click': 'customGoal:YTTHbXeeVJWcWPaDMWOMHTccTBLWCAWdPZKe:1'}
{'class': ['bui-list-item', 

In [462]:
#il faut avoir 256  cell 
print(len(soup2.find_all('a')) )

201


In [463]:
hello=soup2.find_all('a',attrs={'class':"bui-card__header_full_link_wrap"})[0]
print(hello)

<a class="bui-card__header_full_link_wrap" data-lp-ga-click="reviews-targeted-hotel:click-title" href="/hotel/fr/hotel-gabriel.fr.html">
<h3 class="bui-card__title">Hotel Gabriel</h3>
<p class="bui-card__subtitle">
Hôtel dans ce quartier : Le Mont-Saint-Michel
</p>
</a>


In [464]:
#hotel name
testotelname=hello.text.split('\n')[2]
testotelname

''

In [465]:
#test on 1 hotel  before test on all
a=str(hello).split('href=')[1].split( 'title=')[0].split('"')[1]
print(a)

/hotel/fr/hotel-gabriel.fr.html


In [466]:
#how many links
n=len(soup2.find_all('a',attrs={'class':"bui-card__header_full_link_wrap"}))
print("we will get",n, "links")

we will get 9 links


In [467]:
#lets get them
links=[]
names=[]

for i in range(n):
  c=soup2.find_all('a',attrs={'class':"bui-card__header_full_link_wrap"})[i]
  
  #split on c to get link nd put it in list
  names.append(c.text.split('\n')[2])

  links.append('https://www.booking.com'+ str(c).split('href=')[1].split( 'title=')[0].split('"')[1])  #here we add 'https://www.booking.com/' to the hrefs



print(links)
print(names)

['https://www.booking.com/hotel/fr/hotel-gabriel.fr.html', 'https://www.booking.com/hotel/fr/de-la-digue.fr.html', 'https://www.booking.com/hotel/fr/le-relais-du-roy.fr.html', 'https://www.booking.com/hotel/fr/vert.fr.html', 'https://www.booking.com/hotel/fr/les-terrasses-poulard.fr.html', 'https://www.booking.com/hotel/fr/auberge-saint-pierre.fr.html', 'https://www.booking.com/hotel/fr/le-mouton-blanc-le-mont-saint-michel.fr.html', 'https://www.booking.com/hotel/fr/le-relais-saint-michel.fr.html', 'https://www.booking.com/hotel/fr/hotel-saint-aubert.fr.html']
['', '', '', '', '', '', '', '', '']


In [468]:
# #let s add www.book...
# urls=[]
# for i in listvide:
#   urls.append('https://www.booking.com/'+i)
# print(urls[0:4])

In [469]:
#we convert it into dataframe
df=pd.DataFrame()
# df['names']=names[0:3]  #on va prendre que les 3 premiers pour scraper vite apres un page par page...
df['names']=names
df['links']=links
df['lat']=np.nan
df['lon']=np.nan
df['hotel_note']=np.nan
df['description']=np.nan
display(df.head() )


,names,links,lat,lon,hotel_note,description
0,,https://www.booking.com/hotel/fr/hotel-gabriel...,NaN,NaN,NaN,NaN
1,,https://www.booking.com/hotel/fr/de-la-digue.f...,NaN,NaN,NaN,NaN
2,,https://www.booking.com/hotel/fr/le-relais-du-...,NaN,NaN,NaN,NaN
3,,https://www.booking.com/hotel/fr/vert.fr.html,NaN,NaN,NaN,NaN
4,,https://www.booking.com/hotel/fr/les-terrasses...,NaN,NaN,NaN,NaN


# ***now page after page***

In [470]:
for i in df['links']:
  print(i)

https://www.booking.com/hotel/fr/hotel-gabriel.fr.html
https://www.booking.com/hotel/fr/de-la-digue.fr.html
https://www.booking.com/hotel/fr/le-relais-du-roy.fr.html
https://www.booking.com/hotel/fr/vert.fr.html
https://www.booking.com/hotel/fr/les-terrasses-poulard.fr.html
https://www.booking.com/hotel/fr/auberge-saint-pierre.fr.html
https://www.booking.com/hotel/fr/le-mouton-blanc-le-mont-saint-michel.fr.html
https://www.booking.com/hotel/fr/le-relais-saint-michel.fr.html
https://www.booking.com/hotel/fr/hotel-saint-aubert.fr.html


In [471]:
description=[]
hotel_note=[]
lat=[]
lon=[]
p,_=df.shape
p
for i in range(p):

  r=requests.get(df['links'][i])
  print(df['links'][i])
  soup2=BeautifulSoup(r.text,'html.parser')
#=================================================================  

  #to get the decription inside the page
  
  hi=soup2.find('div',{'id':'property_description_content'}).text
  description.append(hi)
 
  # print(description)

#================================================================
#user note
  notebyuser=soup2.find('div',{'class':"b5cd09854e d10a6220b4"}).text
  
  hotel_note.append(notebyuser[0:3])
 

  print(hotel_note[i])
 
#================================================================ 
#gps work 
#i get the code after looking doing this : gps=soup.find('p',{'class':"address address_clean"}) print(gps)
  t1=str(soup2.find('a',{'id':"hotel_header"})).split('bbox="')[1].split(',')[0]
  t2=str(soup2.find('a',{'id':"hotel_header"})).split('bbox="')[1].split(',')[1]

  lat.append(t1)

  lon.append(t2)
 
  print(lat[i])    
  print(lon[i]) 
  #petit condition is on a pas scraper de la 1er page de recherche on recupere les info via la page de chaque hotel 
  if df['names'][i]=='':
    df['names'][i]=soup2.find('h2',{'class':"d2fee87262 pp-header__title"}).text
  else:
    df['names'][i]


https://www.booking.com/hotel/fr/hotel-gabriel.fr.html
7,9
48.5974159971103
-1.53793471039279


<ipython-input-471-e14a69618c91>:44: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



https://www.booking.com/hotel/fr/de-la-digue.fr.html
7,1
48.5989161328681
-1.53814339058278
https://www.booking.com/hotel/fr/le-relais-du-roy.fr.html
7,9
48.5982972879393
-1.53813098690421
https://www.booking.com/hotel/fr/vert.fr.html
8,0
48.5967350697173
-1.53684134303157
https://www.booking.com/hotel/fr/les-terrasses-poulard.fr.html
7,3
48.6173840090681
-1.53761423137484
https://www.booking.com/hotel/fr/auberge-saint-pierre.fr.html
7,9
48.6177225621183
-1.53711820547634
https://www.booking.com/hotel/fr/le-mouton-blanc-le-mont-saint-michel.fr.html
7,0
48.618057567874
-1.537131797379
https://www.booking.com/hotel/fr/le-relais-saint-michel.fr.html
7,9
48.5996218550758
-1.53762208153347
https://www.booking.com/hotel/fr/hotel-saint-aubert.fr.html
7,0
48.5949724181334
-1.5373285546326


In [472]:
df['lat']=lat
df['description']=description
df['lon']=lon
df['hotel_note']=hotel_note

In [473]:
df[0:20]

,names,links,lat,lon,hotel_note,description
0,Hotel Gabriel,https://www.booking.com/hotel/fr/hotel-gabriel...,48.5974159971103,-1.53793471039279,"7,9",\nVous pouvez bénéficier d'une réduction Geniu...
1,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,48.5989161328681,-1.53814339058278,"7,1",\n L'hôtel De La Digue est un établissement tr...
2,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,48.5982972879393,-1.53813098690421,"7,9",\nVous pouvez bénéficier d'une réduction Geniu...
3,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html,48.5967350697173,-1.53684134303157,"8,0",\nVous pouvez bénéficier d'une réduction Geniu...
4,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,48.6173840090681,-1.53761423137484,"7,3",\nVous pouvez bénéficier d'une réduction Geniu...
5,Auberge Saint Pierre,https://www.booking.com/hotel/fr/auberge-saint...,48.6177225621183,-1.53711820547634,"7,9",\n L'Auberge Saint-Pierre occupe une maison à ...
6,Le Mouton Blanc,https://www.booking.com/hotel/fr/le-mouton-bla...,48.618057567874,-1.537131797379,"7,0",\nVous pouvez bénéficier d'une réduction Geniu...
7,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,48.5996218550758,-1.53762208153347,"7,9",\nVous pouvez bénéficier d'une réduction Geniu...
8,Le Saint Aubert,https://www.booking.com/hotel/fr/hotel-saint-a...,48.5949724181334,-1.5373285546326,"7,0","\n Niché dans un écrin de verdure, à seulement..."


In [474]:
len(df['description'].unique())  #

9

In [475]:
df.isnull().any()

names          False
links          False
lat            False
lon            False
hotel_note     False
description    False
dtype: bool

In [476]:
df['lat'][1][0:5]

'48.59'

In [477]:
df['description']=df['description'].apply(lambda x : x.replace('\n','') ) #les \n au debut deleted
df['lat']=df['lat'].apply(lambda x : df['lat'][1][0:5])  # just 2 number after .
df['lon']=df['lon'].apply(lambda x : df['lon'][1][0:5] )

In [478]:
df['lat']=pd.to_numeric(df['lat'])
df['lon']=pd.to_numeric(df['lon'])
type(df['lat'][0])

numpy.float64

In [479]:
df['hotel_note']=df['hotel_note'].apply(lambda x : x.replace(",",'.') ) 

In [480]:
df['hotel_note']=pd.to_numeric(df['hotel_note'])

In [481]:
df.head()

,names,links,lat,lon,hotel_note,description
0,Hotel Gabriel,https://www.booking.com/hotel/fr/hotel-gabriel...,48.59,-1.53,7.9,Vous pouvez bénéficier d'une réduction Genius ...
1,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,48.59,-1.53,7.1,L'hôtel De La Digue est un établissement trad...
2,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,48.59,-1.53,7.9,Vous pouvez bénéficier d'une réduction Genius ...
3,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html,48.59,-1.53,8.0,Vous pouvez bénéficier d'une réduction Genius ...
4,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,48.59,-1.53,7.3,Vous pouvez bénéficier d'une réduction Genius ...


In [482]:
df.to_csv('projetkayak.csv', index=False, encoding='utf-8')

In [483]:
fig = px.scatter_mapbox(df, lat="lat",
                        lon="lon", 
                        color="hotel_note", 
                        zoom=10, 
                        mapbox_style="carto-positron")
fig.show()

In [484]:
dic={
"Paris":'https://www.booking.com/searchresults.fr.html?ss=Paris&ssne=Paris&ssne_untouched=Paris&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAKIxpijBsACAdICJDAwNmExMDRkLTZkMmEtNDA5Yy1iNzNlLWM0N2RhMjQxZTViOdgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1456928&dest_type=city&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure',    
"Mont Saint Michel":'https://www.booking.com/searchresults.fr.html?ss=Le+Mont-Saint-Michel&ssne=Le+Mont-Saint-Michel&ssne_untouched=Le+Mont-Saint-Michel&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAKo8qOjBsACAdICJGRmY2FkMjhkLTY1MWUtNGE2Ny1hMTg5LTQzNDhlNjc1YmEyZNgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=900039327&dest_type=city&group_adults=2&no_rooms=1&group_children=0',
"St Malo":'https://www.booking.com/searchresults.fr.html?ss=St+Malo&ssne=Saint-Malo&ssne_untouched=Saint-Malo&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAKT86OjBsACAdICJDBlZjAxY2Y4LTc4OWQtNDFjZC1hNWQ4LTVhMWM1MTliM2NjM9gCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1466824&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=fr&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=22f37609fbe50123&ac_meta=GhAyMmYzNzYwOWZiZTUwMTIzIAAoATICZnI6B1N0IE1hbG9AAEoAUAA%3D&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure',
"Bayeux":'https://www.booking.com/searchresults.fr.html?ss=Bayeux&ssne=Bayeux&ssne_untouched=Bayeux&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuALw86OjBsACAdICJDFjMGE2N2Q3LTRhOTYtNGFjNC1iNjBlLWY3Mzg5NzEzMDVkZdgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=-1410836&dest_type=city&group_adults=2&no_rooms=1&group_children=0',
"Le Havre":'https://www.booking.com/searchresults.fr.html?ss=Le+Havre&ssne=Le+Havre&ssne_untouched=Le+Havre&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuALw86OjBsACAdICJDFjMGE2N2Q3LTRhOTYtNGFjNC1iNjBlLWY3Mzg5NzEzMDVkZdgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=-1441598&dest_type=city&group_adults=2&no_rooms=1&group_children=0',
}

# "Amiens",
# "Lille",
# "Strasbourg",
# "Chateau du Haut Koenigsbourg",
# "Colmar",
# "Eguisheim",
# "Besancon",
# "Dijon",
# "Annecy",
# "Grenoble",
# "Lyon",
# "Gorges du Verdon",
# "Bormes les Mimosas",
# "Cassis",
# "Marseille",
# "Aix en Provence",
# "Avignon",
# "Uzes",
# "Nimes",
# "Aigues Mortes",
# "Saintes Maries de la mer",
# "Collioure",
# "Carcassonne",
# "Ariege",
# "Toulouse",
# "Montauban",
# "Biarritz",
# "Bayonne",
# "La Rochelle"}

In [507]:
urls=['https://www.booking.com/searchresults.fr.html?ss=Paris&ssne=Paris&ssne_untouched=Paris&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAKIxpijBsACAdICJDAwNmExMDRkLTZkMmEtNDA5Yy1iNzNlLWM0N2RhMjQxZTViOdgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1456928&dest_type=city&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure','https://www.booking.com/searchresults.fr.html?ss=Le+Havre&ssne=Le+Havre&ssne_untouched=Le+Havre&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuALw86OjBsACAdICJDFjMGE2N2Q3LTRhOTYtNGFjNC1iNjBlLWY3Mzg5NzEzMDVkZdgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=-1441598&dest_type=city&group_adults=2&no_rooms=1&group_children=0',
      'https://www.booking.com/searchresults.fr.html?ss=Bayeux&ssne=Bayeux&ssne_untouched=Bayeux&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuALw86OjBsACAdICJDFjMGE2N2Q3LTRhOTYtNGFjNC1iNjBlLWY3Mzg5NzEzMDVkZdgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=-1410836&dest_type=city&group_adults=2&no_rooms=1&group_children=0',
      'https://www.booking.com/searchresults.fr.html?ss=St+Malo&ssne=Saint-Malo&ssne_untouched=Saint-Malo&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAKT86OjBsACAdICJDBlZjAxY2Y4LTc4OWQtNDFjZC1hNWQ4LTVhMWM1MTliM2NjM9gCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1466824&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=fr&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=22f37609fbe50123&ac_meta=GhAyMmYzNzYwOWZiZTUwMTIzIAAoATICZnI6B1N0IE1hbG9AAEoAUAA%3D&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure',
      'https://www.booking.com/searchresults.fr.html?ss=Le+Mont-Saint-Michel&ssne=Le+Mont-Saint-Michel&ssne_untouched=Le+Mont-Saint-Michel&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAKo8qOjBsACAdICJGRmY2FkMjhkLTY1MWUtNGE2Ny1hMTg5LTQzNDhlNjc1YmEyZNgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=900039327&dest_type=city&group_adults=2&no_rooms=1&group_children=0',]

In [521]:
df2=pd.DataFrame()
for url in urls:
    r2=requests.get(url)
    r2.text[0:200]
    # Instanciate BeautifulSoup class
    soup2=BeautifulSoup( r2.text, "html.parser") # Here we used an HTML Parser
    links=[]
    names=[]
    for i in range(n):
        c=soup2.find_all('a',attrs={'class':"bui-card__header_full_link_wrap"})[i]
        names.append(c.text.split('\n')[2])
        links.append('https://www.booking.com'+ str(c).split('href=')[1].split( 'title=')[0].split('"')[1])
    df['names']=names
    df['links']=links
    df['lat']=np.nan
    df['lon']=np.nan
    df['hotel_note']=np.nan
    df['description']=np.nan
    # display(df.head())
    description=[]
    hotel_note=[]
    lat=[]
    lon=[]
    p,_=df.shape
    for i in range(p):
        r=requests.get(df['links'][i])
        
        display(df['links'][i])
        print('--------------------------------------------------------------------------------------------------------------------------------')
        soup2=BeautifulSoup(r.text,'html.parser')
        hi=soup2.find('div',{'id':'property_description_content'}).text
        description.append(hi)
        notebyuser=soup2.find('div',{'class':"b5cd09854e d10a6220b4"}).text
        hotel_note.append(notebyuser[0:3])
        t1=str(soup2.find('a',{'id':"hotel_header"})).split('bbox="')[1].split(',')[0]
        t2=str(soup2.find('a',{'id':"hotel_header"})).split('bbox="')[1].split(',')[1]
        lat.append(t1)
        lon.append(t2)
        # print(lat[i]) #at the end we will have almost the  same lat nd lon bcs we did arround
        if df['names'][i]=='':
          df['names'][i]=soup2.find('h2',{'class':"d2fee87262 pp-header__title"}).text
        else:
         df['names'][i]
    df['lat']=lat
    df['description']=description
    df['lon']=lon
    df['hotel_note']=hotel_note
    df['description']=df['description'].apply(lambda x : x.replace('\n','') ) #les \n au debut deleted
    df['lat']=df['lat'].apply(lambda x : df['lat'][1][0:5])  # just 2 number after .
    df['lon']=df['lon'].apply(lambda x : df['lon'][1][0:5] )
    df['lat']=pd.to_numeric(df['lat'])
    df['lon']=pd.to_numeric(df['lon'])
    df['hotel_note']=df['hotel_note'].apply(lambda x : x.replace(",",'.') ) 
    df['hotel_note']=pd.to_numeric(df['hotel_note'])
    print("Data is Ready now lets get the next one")
    print('===============================================================================================================================')
    df2=pd.concat([df2,df])
    display(df)

print("Data Final")    
display(df2)
    

'https://www.booking.com/hotel/fr/hotelterminuslehavre.fr.html'

--------------------------------------------------------------------------------------------------------------------------------


'https://www.booking.com/hotel/fr/mercure-bassin-du-commerce.fr.html'

--------------------------------------------------------------------------------------------------------------------------------


'https://www.booking.com/hotel/fr/citotel-le-marly.fr.html'

...

'https://www.booking.com/hotel/fr/hotel-saint-aubert.fr.html'

--------------------------------------------------------------------------------------------------------------------------------
Data is Ready now lets get the next one


,names,links,lat,lon,hotel_note,description
0,Hotel Gabriel,https://www.booking.com/hotel/fr/hotel-gabriel...,48.59,-1.53,7.9,Vous pouvez bénéficier d'une réduction Genius ...
1,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,48.59,-1.53,7.1,L'hôtel De La Digue est un établissement trad...
2,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,48.59,-1.53,7.9,Vous pouvez bénéficier d'une réduction Genius ...
3,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html,48.59,-1.53,8.0,Vous pouvez bénéficier d'une réduction Genius ...
4,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,48.59,-1.53,7.3,Vous pouvez bénéficier d'une réduction Genius ...
5,Auberge Saint Pierre,https://www.booking.com/hotel/fr/auberge-saint...,48.59,-1.53,7.9,L'Auberge Saint-Pierre occupe une maison à co...
6,Le Mouton Blanc,https://www.booking.com/hotel/fr/le-mouton-bla...,48.59,-1.53,7.0,Vous pouvez bénéficier d'une réduction Genius ...
7,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,48.59,-1.53,7.9,Vous pouvez bénéficier d'une réduction Genius ...
8,Le Saint Aubert,https://www.booking.com/hotel/fr/hotel-saint-a...,48.59,-1.53,7.0,"Niché dans un écrin de verdure, à seulement 2..."


Data Final


,names,links,lat,lon,hotel_note,description
0,Best Western Plus Le Havre Centre Gare,https://www.booking.com/hotel/fr/hotelterminus...,49.47,0.089,8.4,"Situé dans le centre du Havre, à seulement 10..."
1,Mercure Le Havre Centre Bassin Du Commerce,https://www.booking.com/hotel/fr/mercure-bassi...,49.47,0.089,7.5,Vous pouvez bénéficier d'une réduction Genius ...
2,ibis Styles Le Havre Centre,https://www.booking.com/hotel/fr/citotel-le-ma...,49.47,0.089,6.8,L'ibis Styles Marly Le Havre Centre est situé...
3,Le Richelieu,https://www.booking.com/hotel/fr/le-richelieu-...,49.47,0.089,6.7,Le Citotel Le Richelieu est situé entre la ma...
4,Best Western ARThotel,https://www.booking.com/hotel/fr/art.fr.html,49.47,0.089,7.8,Vous pouvez bénéficier d'une réduction Genius ...
5,Novotel Le Havre Centre Gare,https://www.booking.com/hotel/fr/novotellehavr...,49.47,0.089,7.8,Le Novotel Le Havre est situé sur le front de...
6,Hôtel Les Gens De Mer Le Havre by Popinns,https://www.booking.com/hotel/fr/les-gens-de-m...,49.47,0.089,6.1,Vous pouvez bénéficier d'une réduction Genius ...
7,Birgit Hôtel Le Havre Centre,https://www.booking.com/hotel/fr/kyriadlehavre...,49.47,0.089,8.2,Vous pouvez bénéficier d'une réduction Genius ...
8,B&B HOTEL Le Havre Centre Gare,https://www.booking.com/hotel/fr/b-b-le-havre-...,49.47,0.089,6.8,"Proposant un hébergement 3 étoiles, le B&B HO..."
0,Relais des 3 Pommes,https://www.booking.com/hotel/fr/le-relais-des...,49.25,-0.720,6.9,"Situé à Bayeux, en Normandie, cet hôtel dispo..."
